# Analyse des données RAQDPS

Ce notebook analyse les champs de dépôt RAQDPS et leur relation avec les glaciers.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import geopandas as gpd
from datetime import datetime, timedelta
from pathlib import Path

# Import des modules du projet
from src.raqdps_glacier_coupling import RAQDPSGlacierAnalysis
from src.utils.data_loaders import load_raqdps_data
from src.utils.visualization import plot_time_series

%matplotlib inline
plt.style.use('seaborn-v0_8')

## 1. Chargement d'une période de données RAQDPS

In [ ]:
# Définir la période d'analyse
start_date = datetime(2023, 7, 15, 0)
end_date = datetime(2023, 7, 16, 23)

# Chemins
raqdps_path = Path('../data/raqdps/')

# Variables à analyser
variables = ['BC_dep', 'PM2.5_dep', 'PM10_dep']

print(f"Période: {start_date} à {end_date}")

In [ ]:
# Charger les données (si disponibles)
try:
    ds = load_raqdps_data(
        raqdps_path,
        start_date,
        end_date,
        variables=variables,
        bbox=(-130, 48, -110, 60)  # Ouest canadien
    )
    print("Données chargées avec succès!")
    print(f"Dimensions: {ds.dims}")
    print(f"Variables: {list(ds.data_vars)}")
except Exception as e:
    print(f"Erreur lors du chargement: {e}")
    print("Création de données synthétiques pour la démonstration...")
    
    # Données synthétiques
    times = pd.date_range(start_date, end_date, freq='H')
    lats = np.linspace(48, 60, 50)
    lons = np.linspace(-130, -110, 60)
    
    # Créer des champs synthétiques
    bc_dep = np.random.exponential(1e-10, (len(times), len(lats), len(lons)))
    pm25_dep = np.random.exponential(2e-10, (len(times), len(lats), len(lons)))
    
    ds = xr.Dataset({
        'BC_dep': (['time', 'lat', 'lon'], bc_dep),
        'PM2.5_dep': (['time', 'lat', 'lon'], pm25_dep)
    },
    coords={
        'time': times,
        'lat': lats,
        'lon': lons
    })

## 2. Analyse spatiale des dépôts

In [ ]:
# Moyennes temporelles
bc_mean = ds['BC_dep'].mean(dim='time')
pm25_mean = ds['PM2.5_dep'].mean(dim='time')

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# BC
im1 = bc_mean.plot(ax=axes[0], cmap='YlOrRd', 
                   cbar_kwargs={'label': 'Dépôt BC (kg/m²/s)'})
axes[0].set_title('Dépôt moyen de carbone noir')

# PM2.5
im2 = pm25_mean.plot(ax=axes[1], cmap='YlOrBr',
                     cbar_kwargs={'label': 'Dépôt PM2.5 (kg/m²/s)'})
axes[1].set_title('Dépôt moyen de PM2.5')

plt.tight_layout()

## 3. Extraction pour des glaciers spécifiques

In [ ]:
# Positions de glaciers d'intérêt
glaciers_of_interest = [
    {'name': 'Athabasca', 'lat': 52.185, 'lon': -117.252},
    {'name': 'Saskatchewan', 'lat': 52.150, 'lon': -117.186},
    {'name': 'Columbia', 'lat': 52.171, 'lon': -117.329},
    {'name': 'Peyto', 'lat': 51.670, 'lon': -116.533}
]

# Extraire les séries temporelles
glacier_series = {}

for glacier in glaciers_of_interest:
    # Sélectionner le point le plus proche
    point_data = ds.sel(
        lat=glacier['lat'],
        lon=glacier['lon'],
        method='nearest'
    )
    
    glacier_series[glacier['name']] = {
        'BC_dep': point_data['BC_dep'].values,
        'PM2.5_dep': point_data['PM2.5_dep'].values,
        'time': point_data.time.values
    }

In [ ]:
# Comparer les séries temporelles
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

for glacier_name, data in glacier_series.items():
    # BC
    axes[0].plot(data['time'], data['BC_dep'], 
                label=glacier_name, linewidth=2)
    
    # PM2.5
    axes[1].plot(data['time'], data['PM2.5_dep'], 
                label=glacier_name, linewidth=2)

axes[0].set_ylabel('Dépôt BC (kg/m²/s)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_ylabel('Dépôt PM2.5 (kg/m²/s)')
axes[1].set_xlabel('Temps')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.suptitle('Comparaison des dépôts sur différents glaciers')
plt.tight_layout()

## 4. Analyse des conditions météorologiques

In [ ]:
# Ajouter des variables météo synthétiques
ds['u_10m'] = xr.DataArray(
    np.random.normal(5, 2, ds['BC_dep'].shape),
    dims=['time', 'lat', 'lon'],
    coords=ds.coords
)

ds['v_10m'] = xr.DataArray(
    np.random.normal(2, 1, ds['BC_dep'].shape),
    dims=['time', 'lat', 'lon'],
    coords=ds.coords
)

# Calculer la vitesse et direction du vent
wind_speed = np.sqrt(ds['u_10m']**2 + ds['v_10m']**2)
wind_dir = np.arctan2(ds['v_10m'], ds['u_10m']) * 180 / np.pi

In [ ]:
# Analyser la relation vent-dépôt pour Athabasca
athabasca_wind = wind_speed.sel(
    lat=52.185, 
    lon=-117.252, 
    method='nearest'
)

athabasca_bc = ds['BC_dep'].sel(
    lat=52.185, 
    lon=-117.252, 
    method='nearest'
)

# Scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(athabasca_wind, athabasca_bc, alpha=0.6)
plt.xlabel('Vitesse du vent (m/s)')
plt.ylabel('Dépôt BC (kg/m²/s)')
plt.title('Relation vent-dépôt au glacier Athabasca')
plt.grid(True, alpha=0.3)

# Ajouter une ligne de tendance
z = np.polyfit(athabasca_wind, athabasca_bc, 1)
p = np.poly1d(z)
plt.plot(sorted(athabasca_wind.values), 
         p(sorted(athabasca_wind.values)), 
         'r--', linewidth=2)

## 5. Calcul des dépôts cumulatifs

In [ ]:
# Convertir en DataFrame pour faciliter les calculs
athabasca_df = pd.DataFrame({
    'time': ds.time.values,
    'BC_dep': athabasca_bc.values,
    'PM25_dep': ds['PM2.5_dep'].sel(
        lat=52.185, lon=-117.252, method='nearest'
    ).values
})
athabasca_df.set_index('time', inplace=True)

# Calculer les cumulatifs
# Conversion approximative: 1 heure de dépôt
athabasca_df['BC_hourly'] = athabasca_df['BC_dep'] * 3600
athabasca_df['BC_cumul'] = athabasca_df['BC_hourly'].cumsum()
athabasca_df['BC_24h'] = athabasca_df['BC_hourly'].rolling('24H').sum()

print("Dépôts cumulatifs calculés")

In [ ]:
# Visualiser les cumulatifs
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Dépôt horaire
axes[0].plot(athabasca_df.index, athabasca_df['BC_hourly'], 
            'k-', alpha=0.5, label='Horaire')
axes[0].plot(athabasca_df.index, athabasca_df['BC_24h'], 
            'k-', linewidth=2, label='Moyenne 24h')
axes[0].set_ylabel('Dépôt BC (kg/m²/h)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Cumulatif
axes[1].plot(athabasca_df.index, athabasca_df['BC_cumul'], 
            'r-', linewidth=2)
axes[1].set_ylabel('Dépôt BC cumulatif (kg/m²)')
axes[1].set_xlabel('Temps')
axes[1].grid(True, alpha=0.3)

plt.suptitle('Dépôts de carbone noir - Glacier Athabasca')
plt.tight_layout()

## 6. Export des résultats

In [ ]:
# Sauvegarder les séries temporelles
output_dir = Path('../results/raqdps_analysis/')
output_dir.mkdir(parents=True, exist_ok=True)

# Sauvegarder le DataFrame Athabasca
athabasca_df.to_csv(output_dir / 'athabasca_deposition_series.csv')

# Sauvegarder les statistiques
stats = {
    'glacier': 'Athabasca',
    'period': f"{start_date} to {end_date}",
    'BC_total_kg_m2': athabasca_df['BC_cumul'].iloc[-1],
    'BC_max_hourly': athabasca_df['BC_hourly'].max(),
    'BC_mean_hourly': athabasca_df['BC_hourly'].mean(),
    'n_hours': len(athabasca_df)
}

stats_df = pd.DataFrame([stats])
stats_df.to_csv(output_dir / 'deposition_statistics.csv', index=False)

print("Résultats sauvegardés dans:", output_dir)